<a href="https://colab.research.google.com/github/curyumafu/sakana/blob/main/InceptionResNetV2(ImageNet)%E4%BA%88%E6%B8%AC2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

セル 1: インポートと環境変数の設定

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
tf.get_logger().setLevel('ERROR')


セル 2: 他のインポート

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import base64
from io import BytesIO
import numpy as np
from PIL import Image, ImageFont, ImageDraw
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input, decode_predictions
from keras.preprocessing import image
import IPython
from google.colab import output
from google.colab.patches import cv2_imshow
import cv2
from IPython.display import display, Javascript
from google.colab.output import eval_js


Mounted at /content/drive


セル 3: 定数の定義

In [ ]:
FONT_PATH = '/content/drive/MyDrive/実験III_AI演習（ImageNet）2023/ipaexg.ttf'
FONT_SIZE = 20
DISPLAY_SIZE = 299


セル 4: ディレクトリの変更とモジュールのインポート

In [ ]:
_colab_dir = "/content/drive/MyDrive/実験III_AI演習（ImageNet）2023"
os.chdir(_colab_dir)

import to_japanese_ilsvrc2012 as toj
translator = toj.Ilsvrc2012Japanese()


セル 5: モデルのロード

In [ ]:
model = InceptionResNetV2(weights='imagenet')


225209952/225209952 [==============================] - 9s 0us/step


セル 6: 関数の定義

In [ ]:
def img_add_msg(img, messages):
    # フォントの設定
    font = ImageFont.truetype(FONT_PATH, FONT_SIZE)
    # 描画オブジェクトの作成
    draw = ImageDraw.Draw(img)
    # メッセージのリストから各メッセージを画像に描画
    for i, message in enumerate(messages):
        draw.text((0, 10 + i * 20), message, font=font, fill=(255, 255, 255, 0))
    return img

def run(img_str):
    # 画像のデコード
    decimg = base64.b64decode(img_str.split(',')[1], validate=True)
    decimg = Image.open(BytesIO(decimg))
    decimg = np.array(decimg, dtype=np.uint8)
    decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)

    # 画像の前処理
    trim_frame = decimg[0 : DISPLAY_SIZE, 0 : DISPLAY_SIZE]
    x = image.img_to_array(trim_frame)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # 画像の予測
    preds = model.predict(x, verbose=0)
    top_preds = decode_predictions(preds, top=5)[0]

    # メッセージの生成
    if int(top_preds[0][2] * 100) >= 50:
        messages = [f"TOP{i+1} : {translator.convert(str(pred[1]))} : {int(pred[2]*100)}%" for i, pred in enumerate(top_preds)]
    else:
        messages = ["NONE"]

    # 画像にメッセージを追加
    img = Image.fromarray(trim_frame)
    img = img_add_msg(img, messages)

    # 画像をbase64形式に変換して返す
    img = np.array(img)
    out_img = cv2.resize(img, (DISPLAY_SIZE, DISPLAY_SIZE))
    _, encimg = cv2.imencode(".jpg", out_img, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
    img_str = "data:image/jpeg;base64," + base64.b64encode(encimg.tobytes()).decode('utf-8')
    return IPython.display.JSON({'img_str': img_str})

# コールバックの登録
output.register_callback('notebook.run', run)

def use_cam(quality=0.8):
  # JavaScriptコードの定義：ウェブカメラの利用
  js = Javascript('''
    async function useCam(quality) {
      const div = document.createElement('div');
      document.body.appendChild(div);
      //video element
      const video = document.createElement('video');
      video.style.display = 'None';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      //canvas for display. frame rate is depending on display size and jpeg quality.
      display_size = 299
      const src_canvas = document.createElement('canvas');
      src_canvas.width  = display_size;
      src_canvas.height = display_size;;
      const src_canvasCtx = src_canvas.getContext('2d');
      src_canvasCtx.translate(src_canvas.width, 0);
      src_canvasCtx.scale(-1, 1);
      div.appendChild(src_canvas);

      const dst_canvas = document.createElement('canvas');
      dst_canvas.width  = src_canvas.width;
      dst_canvas.height = src_canvas.height;
      const dst_canvasCtx = dst_canvas.getContext('2d');
      div.appendChild(dst_canvas);

      //exit button
      const btn_div = document.createElement('div');
      document.body.appendChild(btn_div);
      const exit_btn = document.createElement('button');
      exit_btn.textContent = 'Exit';
      var exit_flg = true
      exit_btn.onclick = function() {exit_flg = false};
      btn_div.appendChild(exit_btn);

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      var send_num = 0
      // loop
      _canvasUpdate();
      async function _canvasUpdate() {
            src_canvasCtx.drawImage(video, 0, 0, video.videoWidth, video.videoHeight, 0, 0, src_canvas.width, src_canvas.height);
            if (send_num<1){
                send_num += 1
                const img = src_canvas.toDataURL('image/jpeg', quality);
                const result = google.colab.kernel.invokeFunction('notebook.run', [img], {});
                result.then(function(value) {
                    parse = JSON.parse(JSON.stringify(value))["data"]
                    parse = JSON.parse(JSON.stringify(parse))["application/json"]
                    parse = JSON.parse(JSON.stringify(parse))["img_str"]
                    var image = new Image()
                    image.src = parse;
                    image.onload = function(){dst_canvasCtx.drawImage(image, 0, 0)}
                    send_num -= 1
                })
            }
            if (exit_flg){
                requestAnimationFrame(_canvasUpdate);
            }else{
                stream.getVideoTracks()[0].stop();
            }
      };
    }
    ''')
  display(js)
  # ウェブカメラの起動
  data = eval_js('useCam({})'.format(quality))


セル 7: ウェブカメラの利用

In [ ]:
use_cam()


<IPython.core.display.Javascript object>

35363/35363 [==============================] - 0s 0us/step
